# Data

Предварительно подгружал все из последнего проекта

In [1]:
!pip install tensorflow == 2.3
# аугментации изображений
!pip install albumentations
!pip install pymorphy2
!pip install pymorphy2-dicts

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import seaborn as sns
import random
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import pymorphy2
from nltk.corpus import stopwords
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers.schedules import *
import albumentations as a

# plt
import matplotlib.pyplot as plt
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

Метрика MAPE

In [3]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

Метрика MAE:

In [4]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred-y_true))

In [5]:
from sklearn.metrics import mean_squared_error
import math

Фиксируем рандом сид

In [6]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Подгружаем дата-сеты для обучения: Книги и настолки

In [7]:
DATA_DIR = '../input/books-for-pretrain/'
data = pd.read_csv(DATA_DIR + 'books_df.csv')

посмотрим, что получилось:

In [8]:
data.head()

# Очистка и обработка данных

Удалим мусор из ячеек:

In [ ]:
data['description']=data['description'].apply(lambda x:x.replace('\r',''))

In [ ]:
data['Name']=data['Name'].apply(lambda x:x.replace('\r',''))

In [ ]:
data['description']=data['description'].apply(lambda x:x.replace('\n',''))

In [ ]:
data['Name']=data['Name'].apply(lambda x:x.replace('\n',''))

In [ ]:
data['description']=data['description'].apply(lambda x:x.replace('Читать дальше…',''))

Результат:

In [ ]:
data.head()

# Небольшой Feature engeneering

Выделим несколько фич:

In [ ]:
data['desc_length']=data['description'].apply(lambda x:len(x))

In [ ]:
data['name_length']=data['Name'].apply(lambda x:len(x))

Результат:

In [ ]:
data.head()

Проверим количество строк:

In [ ]:
len(data)

Проверим пропуски:

In [ ]:
data.isna().sum()

Удалим мусор из target'a

In [ ]:
data['price']=data['price'].apply(lambda x:x.replace('\xa0',''))

Переведем в int:

In [ ]:
data['price'] = data['price'].apply(lambda x: int(x))

Предобработка данных:

In [ ]:
def preproc_data(df_input):

    df_output = df_input.copy()

    # Удалим неиспользуемые столбцы
    df_output.drop(['description','Name'], axis=1, inplace=True)
    
    scaler = StandardScaler()
    for column in num_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features:
        df_output[column], _ = pd.factorize(df_output[column])

    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features, dummy_na=False)

    return df_output

Используемые для обучения столбцы:

In [ ]:
num_features=['name_length','desc_length']

In [ ]:
cat_features=['author','more_info']

Результат:

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

# Train-test split

Обработка и разделение данных на обучающую и валидационную выборки:

In [ ]:
y = df_preproc.price.values     # наш таргет
X = df_preproc.drop(['price'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25,shuffle=True, random_state=RANDOM_SEED)

# Примитивный бейзлайн

In [ ]:
print(f"TEST mae: {(mae(y_test, np.mean(y_train))):0.2f}")

In [ ]:
print(f"TEST mape: {(mape(y_test, np.mean(y_train)))*100:0.2f}%")

Как видно, погрешность довольно сильная.

# Линейная регрессия

Попробуем использовать Линейную регрессию для определения target'a:

In [ ]:
linreg = LinearRegression().fit(X_train, np.log(y_train))
test_predict = linreg.predict(X_test)

In [ ]:
test_predict

In [ ]:
plt.hist(test_predict)

In [ ]:
test_predict.max()

Уберем выбросы и посчитаем 

In [ ]:
y_test_new=[]
pred=[]

In [ ]:
for i in range(0,len(y_test)-1):
    if np.exp(test_predict[i])!=np.inf:
        y_test_new.append(y_test[i])
        pred.append(np.exp(test_predict[i]))

In [ ]:
y_test_new1 = np.array(y_test_new)

In [ ]:
y_test_new1

In [ ]:
pred1=np.array(pred)

In [ ]:
print(f"TEST mape: {(mape(y_test_new1,pred1))*100:0.2f}%")

In [ ]:
print(f"TEST mae: {(mae(y_test_new1, pred1)):0.2f}")

In [ ]:
print(f"TEST rmse: {math.sqrt(mean_squared_error(y_test_new1, pred1)):0.2f}")

Погрешность существенно уменьшилась, но возникли непредвиденные трудности при работе с выбросами.

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rand_forest=RandomForestRegressor(n_estimators=500,
                                           n_jobs=-1,
                                           max_depth=15,
                                           max_features='log2',
                                           random_state=RANDOM_SEED,
                                           oob_score=True).fit(X_train, np.log(y_train))

In [ ]:
rand_predict=np.exp(rand_forest.predict(X_test))

In [ ]:
print(f"TEST mape: {(mape(y_test,rand_predict))*100:0.2f}%")

In [ ]:
print(f"TEST mae: {(mae(y_test, rand_predict)):0.2f}")

Погрешность вышла хуже чем при линейной регрессии

In [ ]:
print(f"TEST rmse: {math.sqrt(mean_squared_error(y_test, rand_predict)):0.2f}")

# Catboost

Попробуем использовать CatBoost:

In [ ]:
model = CatBoostRegressor(iterations=60000,  # Количество итераций
                          # depth=10,
                          learning_rate = 0.1,
                          random_seed=RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,  # Прерывает выполнение, если нет улучшения 500 итераций
                          task_type='GPU',
                          )
# обучим модель
model.fit(X_train, np.log(y_train),
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=100,
          use_best_model=True,
          # plot=True
          )

In [ ]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

In [ ]:
print(f"TEST mae: {(mae(y_test, test_predict_catboost)):0.2f}")

In [ ]:
print(f"TEST rmse: {math.sqrt(mean_squared_error(y_test, rand_predict)):0.2f}")

Погрешность еще немного уменьшилась. Посмотрим, что получится при использовании простой нейронной сети из нескольких Dense-слоев.

# MLP сеть

In [ ]:
model_ml = Sequential()
model_ml.add(L.Dense(1024, input_dim=X_train.shape[1], activation="relu"))
model_ml.add(L.Dense(512, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-7), activation="relu"))
model_ml.add(L.Dense(1, activation="linear"))

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.001)
model_ml.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model_ml.fit(X_train, (y_train),
                       batch_size=512,  # размер батча
                       epochs=500,  # количество эпох для обучения
                       validation_data=(X_test, (y_test)),  # данные для валидации
                       callbacks=callbacks_list,  # список  наших коллбэков
                       verbose=1,  # параметр, отвечающий за выведение прогресс-бара
                       )

In [ ]:
test_predict_nn1 = model_ml.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [ ]:
print(f"TEST mae: {(mae(y_test, test_predict_nn1[:,0])):0.2f}")

In [ ]:
print(f"TEST rmse: {math.sqrt(mean_squared_error(y_test, test_predict_nn1[:,0])):0.2f}")

# NLP+MLP сеть

## Предобработка текста

In [57]:
morphy = pymorphy2.MorphAnalyzer()
df_NLP = data.copy()

In [56]:
# Паттерн с символами
trash_sym = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-–»«•∙·✔➥●☛“”°№₽®]+"

# функция для лемматизации текста:


def lemma(text):
    text = text.lower()  # понижаем регистр
    text = re.sub(trash_sym, ' ', text)  # удаляем символы из паттерна
    strings = []  # создаем массив, в котором будут храниться лемматизированные строки
    for wrd in text.split():  # берем слово из строки
        wrd = wrd.strip()  # убираем пробелы до и после слова
        wrd = morphy.normal_forms(wrd)[0]  # приводим к нормальной форме
        strings.append(wrd)  # добавляем слово в строку массива
    return ' '.join(strings)  # вернем значения, разделив пробелами

In [ ]:
strings_set = []
strings_set = df_NLP.apply(
    lambda df_NLP: lemma(df_NLP.description), axis=1)

In [58]:
russian_stopwords = stopwords.words("russian")

In [59]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]


# применим функцию к нашим лемматизированым строкам слов
str_without_stop = [lineWithoutStopWords(line) for line in strings_set]

In [ ]:
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [ ]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

In [ ]:
%%time
text_train_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape,
      text_test_sequences.shape )

## NLP

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_package"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LayerNormalization())
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(128, activation="relu"))
model_nlp.add(L.LSTM(64,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

## MLP

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
model_mlp.add(L.Dropout(0.25))

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(32, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)
# Проведем компиляцию модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=15, restore_best_weights=True,)
lr_scheduler = ReduceLROnPlateau(monitor='val_MAPE',
                                 factor=0.5,  # уменьшим lr в 2 раза
                                 patience=10,  # если нет улучшения через 2 эпохи - уменьшить lr
                                 min_lr=0.00001,  # минимальная скорость обучения
                                 verbose=1,  # выводить сообщения об уменьшении скорости
                                 mode='auto')  # выбранный способ отслеживания метрики
callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500,
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                    )

In [ ]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
print(f"TEST mae: {(mae(y_test, test_predict_nn2[:,0])):0.2f}")

In [ ]:
print(f"TEST rmse: {math.sqrt(mean_squared_error(y_test, test_predict_nn2[:,0])):0.2f}")

# TabNet с K-Fold

In [ ]:
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
X_train_new=X_train.values

In [ ]:
y_train_new=np.log(y_train).reshape(-1,1)

In [ ]:
X_test_new=X_test.values

In [ ]:
y_test_new=np.log(y_test).reshape(-1,1)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5, random_state=RANDOM_SEED, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X_train1):
    X_train1, X_valid1 = X_train_new[train_index], X_train_new[test_index]
    y_train1, y_valid1 = y_train_new[train_index], y_train_new[test_index]
    regressor = TabNetRegressor(verbose=1,seed=RANDOM_SEED)
    regressor.fit(X_train=X_train1, y_train=y_train1,
              eval_set=[(X_valid1, y_valid1)],
              patience=1, max_epochs=5,
              eval_metric=['mae'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.exp(regressor.predict(X_test_new)))

predictions = np.mean(predictions_array,axis=0)

In [ ]:
print(f"TEST mape: {(mape(y_test, predictions))*100:0.2f}%")

In [ ]:
print(f"TEST rmse: {math.sqrt(mean_squared_error(y_test, predictions)):0.2f}")

# TabNet обычный

In [ ]:
regressor1=TabNetRegressor(verbose=1,seed=RANDOM_SEED)

In [ ]:
regressor1.fit(X_train=X_train.values, y_train=np.log(y_train.reshape(-1,1)),
              eval_set=[(X_test.values, np.log(y_test.reshape(-1,1)))],
              patience=5, max_epochs=20,
              eval_metric=['mae'],
              batch_size=1024, 
              virtual_batch_size=128)

In [ ]:
predict_stream=np.exp(regressor1.predict(X_test.values))

In [ ]:
print(f"TEST mape: {(mape(y_test, predict_stream))*100:0.2f}%")

# Второй дата-сет (целевой)

Данные

In [9]:
DATA_DIR1 = '../input/tabletop-games/'
data_games = pd.read_csv(DATA_DIR1 + 'games_hg.csv')

In [10]:
data_games1=data_games.copy()

In [11]:
data_games1=data_games1[data_games1.price !='Цена скоро будет']

In [36]:
data_games1=data_games1.reset_index(drop=True)

In [37]:
data_games1.tail()

# Небольшая предобработка

In [38]:
data_games1['players']=data_games1['players'].apply(lambda x:x.split(' ')[0])

In [39]:
data_games1['time']=data_games1['time'].apply(lambda x:x.split(' ')[0])

In [40]:
data_games1['price']=data_games1['price'].apply(lambda x:x.replace('бон.',''))
data_games1['price']=data_games1['price'].apply(lambda x:x.replace('₽',''))

In [41]:
data_games1['price']=data_games1['price'].apply(lambda x:x.replace('\xa0',''))

In [44]:
data_games1['price'] = data_games1['price'].apply(lambda x: int(x))

In [45]:
data_games1['description']=data_games1['description'].apply(lambda x:x.replace('\r',''))
data_games1['description']=data_games1['description'].apply(lambda x:x.replace('\n',''))
data_games1['description']=data_games1['description'].apply(lambda x:x.replace('\t',''))
data_games1['package']=data_games1['package'].apply(lambda x:x.replace('\r',''))
data_games1['package']=data_games1['package'].apply(lambda x:x.replace('\n',''))
data_games1['package']=data_games1['package'].apply(lambda x:x.replace('\t',''))

# Feature engeneering

In [20]:
data_games1['cards']=data_games1['package'].apply(lambda x:1 if 'карт' in x else 0)

In [21]:
data_games1['figures']=data_games1['package'].apply(lambda x:1 if 'фигур' in x else 0)

In [22]:
data_games1['pack_length']=data_games1['package'].apply(lambda x:len(x))

In [23]:
data_games1['desc_length']=data_games1['description'].apply(lambda x:len(x))

In [46]:
data_games1.head()

In [47]:
cat_features1=['players','time','age','manufacturer','cards','figures']
num_features1=['pack_length','desc_length']

In [48]:
def preproc_data1(df_input):

    df_output = df_input.copy()

    # Удалим неиспользуемые столбцы
    df_output.drop(['description', 'package','gameName',
                    'image'], axis=1, inplace=True)

    scaler = StandardScaler()
    for column in num_features1:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features1:
        df_output[column], _ = pd.factorize(df_output[column])

    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features1, dummy_na=False)

    return df_output

In [49]:
df_preproc1 = preproc_data1(data_games1)
df_preproc1.head(10)

In [50]:
y1 = df_preproc1.price.values     # наш таргет
X1 = df_preproc1.drop(['price'], axis=1)

In [51]:
X_train_games, X_test_games, y_train_games, y_test_games = train_test_split(
    X1, y1, test_size=0.15,shuffle=True, random_state=RANDOM_SEED)

# Бейзлайн

In [ ]:
print(f"TEST mape: {(mape(y_test_games, np.mean(y_train_games)))*100:0.2f}%")

# Линейная регрессия

In [ ]:
linreg = LinearRegression().fit(X_train_games, np.log(y_train_games))
predict_linreg = np.exp(linreg.predict(X_test_games))

In [ ]:
y_test_new1=[]
linreg_pred=[]

In [ ]:
for i in range(0,len(y_test_games)-1):
    if predict_linreg[i]!=np.inf:
        y_test_new1.append(y_test_games[i])
        linreg_pred.append(predict_linreg[i])

In [ ]:
y_test_new1=np.array(y_test_new1)
linreg_pred=np.array(linreg_pred)

In [ ]:
print(f"TEST mape: {(mape(y_test_new1,linreg_pred))*100:0.2f}%")

Погрешность упала, по сравнению с бейзлайном

# RandomForest

In [ ]:
game_forest=RandomForestRegressor(n_estimators=500,
                                           n_jobs=-1,
                                           max_depth=15,
                                           max_features='log2',
                                           random_state=RANDOM_SEED,
                                           oob_score=True).fit(X_train_games, np.log(y_train_games))

rf_predict_games=np.exp(game_forest.predict(X_test_games))

In [ ]:
print(f"TEST mape: {(mape(y_test_games,rf_predict_games))*100:0.2f}%")

Погрешность вышла немного хуже, чем у линейной регрессии, но лучше, чем бейзлайн

# Catboost

In [ ]:
cb_model = CatBoostRegressor(iterations=60000,  # Количество итераций
                             learning_rate = 0.1,
                             random_seed=RANDOM_SEED,
                             eval_metric='MAPE',
                             custom_metric=['RMSE', 'MAE'],
                             od_wait=500,  # Прерывает выполнение, если нет улучшения 500 итераций
                             )
# обучим модель
cb_model.fit(X_train_games, np.log(y_train_games),
             eval_set=(X_test_games, np.log(y_test_games)),
             verbose_eval=100,
             use_best_model=True,
             )

In [ ]:
game_predict_catboost = np.exp(cb_model.predict(X_test_games))

In [ ]:
print(f"TEST mape: {(mape(y_test_games, game_predict_catboost))*100:0.2f}%")

Метрика еще немного улучшилась, что не может не радовать

# MLP сеть

In [ ]:
model_ml1 = Sequential()
model_ml1.add(L.Dense(1024, input_dim=X_train_games.shape[1], activation="relu"))
model_ml1.add(L.Dense(512, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml1.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_ml1.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-7), activation="relu"))
model_ml1.add(L.Dense(1, activation="linear"))

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adamax(0.001)
model_ml1.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history1 = model_ml1.fit(X_train_games, y_train_games,
                        batch_size=512,  # размер батча
                        epochs=500,  # количество эпох для обучения
                        validation_data=(X_test_games, y_test_games),  # данные для валидации
                        callbacks=callbacks_list,  # список  наших коллбэков
                        verbose=1,  # параметр, отвечающий за выведение прогресс-бара
                        )

In [ ]:
game_predict_nn1 = model_ml1.predict(X_test_games)
print(f"TEST mape: {(mape(y_test_games, game_predict_nn1[:,0]))*100:0.2f}%")

Метрика еще улучшилась, посмотрим что покажут другие модели.

# MLP+NLP+NLP

Обработка текста

In [54]:
df_NLP1 = data_games1.copy()

In [60]:
strings_set1 = []
strings_set1 = df_NLP1.apply(
    lambda df_NLP1: lemma(df_NLP1.description), axis=1)

In [61]:
strings_set2 = []
strings_set2 = df_NLP1.apply(
    lambda df_NLP1: lemma(df_NLP1.package), axis=1)

In [62]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]

In [63]:
# применим функцию к нашим лемматизированым строкам слов
str_without_stop1 = [lineWithoutStopWords(line) for line in strings_set1]
str_without_stop2 = [lineWithoutStopWords(line) for line in strings_set2]

In [64]:
text_train_games = data_games1.package.iloc[X_train_games.index]
text_test_games = data_games1.package.iloc[X_test_games.index]

In [65]:
text_train_games1 = data_games1.description.iloc[X_train_games.index]
text_test_games1 = data_games1.description.iloc[X_test_games.index]

In [66]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [67]:
%%time
tokenize1 = Tokenizer(num_words=MAX_WORDS)
tokenize1.fit_on_texts(str_without_stop1)

In [68]:
%%time
tokenize2 = Tokenizer(num_words=MAX_WORDS)
tokenize2.fit_on_texts(str_without_stop2)

In [77]:
%%time
text_train_sequences1 = sequence.pad_sequences(
    tokenize1.texts_to_sequences(text_train_games), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences1 = sequence.pad_sequences(
    tokenize1.texts_to_sequences(text_test_games), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences1.shape,
      text_test_sequences1.shape )

In [78]:
%%time
text_train_sequences2 = sequence.pad_sequences(
    tokenize2.texts_to_sequences(text_train_games1), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences2 = sequence.pad_sequences(
    tokenize2.texts_to_sequences(text_test_games1), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences2.shape,
      text_test_sequences2.shape )

# NLP1 сеть

In [72]:
model_nlp1 = Sequential()
model_nlp1.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_package1"))
model_nlp1.add(L.Embedding(len(tokenize1.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp1.add(L.LayerNormalization())
model_nlp1.add(L.LSTM(256, return_sequences=True))
model_nlp1.add(L.Dropout(0.5))
model_nlp1.add(L.Dense(128, activation="relu"))
model_nlp1.add(L.LSTM(64,))
model_nlp1.add(L.Dropout(0.25))
model_nlp1.add(L.Dense(64, activation="relu"))
model_nlp1.add(L.Dropout(0.25))

# NLP2 сеть

In [73]:
model_nlp2 = Sequential()
model_nlp2.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_package2"))
model_nlp2.add(L.Embedding(len(tokenize2.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp2.add(L.LayerNormalization())
model_nlp2.add(L.LSTM(256, return_sequences=True))
model_nlp2.add(L.Dropout(0.5))
model_nlp2.add(L.Dense(128, activation="relu"))
model_nlp2.add(L.LSTM(64,))
model_nlp2.add(L.Dropout(0.25))
model_nlp2.add(L.Dense(64, activation="relu"))
model_nlp2.add(L.Dropout(0.25))

# MLP сеть

In [74]:
model_mlp1 = Sequential()
model_mlp1.add(L.Dense(512, input_dim=X_train_games.shape[1], activation="relu"))
model_mlp1.add(L.Dropout(0.5))
model_mlp1.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_mlp1.add(L.Dropout(0.5))
model_mlp1.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
model_mlp1.add(L.Dropout(0.25))

In [75]:
combinedInput = L.concatenate([model_nlp1.output,model_nlp2.output, model_mlp1.output])
# being our regression head
head = L.Dense(32, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp1.input,model_nlp2.input, model_mlp1.input], outputs=head)

In [76]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=15, restore_best_weights=True,)
lr_scheduler = ReduceLROnPlateau(monitor='val_MAPE',
                                 factor=0.5,  # уменьшим lr в 2 раза
                                 patience=10,  # если нет улучшения через 2 эпохи - уменьшить lr
                                 min_lr=0.00001,  # минимальная скорость обучения
                                 verbose=1,  # выводить сообщения об уменьшении скорости
                                 mode='auto')  # выбранный способ отслеживания метрики
callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [81]:
optimizer = tf.keras.optimizers.Adamax(0.001)
# Проведем компиляцию модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model.fit([text_train_sequences1,text_train_sequences2, X_train_games], y_train_games,
                    batch_size=512,
                    epochs=500,
                    validation_data=([text_test_sequences1,text_test_sequences2, X_test_games], y_test_games),
                    callbacks=callbacks_list
                    )

In [ ]:
game_predict_nn2 = model.predict([text_test_sequences1,text_test_sequences2, X_test_games])
print(f"TEST mape: {(mape(y_test, game_predict_nn2[:,0]))*100:0.2f}%")